**Reference**: <https://adv-r.hadley.nz/conditions.html>

In [113]:
library(rlang)
library(tidyverse)
library(glue)


Attaching package: 'glue'


The following object is masked from 'package:dplyr':

    collapse




# 1. Introduction

3 conditions:
- Error: most severe; they indicate that there is no way for a function to continue and execution must stop.
- Warning: somewhat in between errors and message, and typically indicate that something has gone wrong but the function has been able to at least partially recover.
- Message: the mildest; they are way of informing users that some action has been performed on their behalf.

Big picture:
- Signal conditions: Something unusual is happening from the function you create
- Handling conditions: handle conditions signalled from your function

# 2. Signal Condition

### 2.1 Signal an error

Once an error occured, the rest of the function call will be ignored:


In [19]:
f <- function() {
    stop('something wrong')
    print('this message will be ignored')
}
f()

ERROR: Error in f(): something wrong


 By default, the error message includes the call:

In [14]:
f <- function() g()
g() <- function() stop('ERROR!!!')
g()

ERROR: Error in g() <- function() stop("ERROR!!!"): invalid (NULL) left side of assignment


This is not useful because we can use **`traceback`**, so it's better to turn it off

In [16]:
h <- function() stop('ERROR!!!!!', call. = F)
h()

ERROR: Error: ERROR!!!!!


In [8]:
stop('DivisionByZero')

ERROR: Error in eval(expr, envir, enclos): DivisionByZero


In [17]:
# Automatically does not includes the call
rlang::abort('DivisionByZero')

ERROR: Error: DivisionByZero


### 2.2 Signal a warning

You can have multiple warning in a function call:

In [20]:
f <- function() {
    warning('One')
    print('I')
    warning('Two')
    print('Love')
}
f()

Warning message in f():
"One"


[1] "I"


Warning message in f():
"Two"


[1] "Love"


In [10]:
# signal a warning
warning('Something is wrong')

Warning message in eval(expr, envir, enclos):
"Something is wrong"


In [11]:
rlang::warn('Something is wrong')

Warning message:
"Something is wrong"


### 2.3 Signal a message

In [12]:
# signal a message
message('The function you used is deprecated')

The function you used is deprecated



In [13]:
rlang::inform('The function you used is deprecated')

The function you used is deprecated



# 3. Ignoring Condition

In [25]:
# Ignore error
f <- function() {
    try(log('a'))
    10
}
f()

Error in log("a") : non-numeric argument to mathematical function


[1] 10

In [26]:
# Ignore warning
# input is an expression
suppressWarnings({
    warning('whatever')
    1
})

[1] 1

In [28]:
# Ignore message
suppressMessages({
    message('Computing....')
    2
})

[1] 2

# 4. Handling Conditions

>These functions will change the default behavior of condition
For example, the default behavior of error condition is to stop the program. But what if instead of stop the program, we want to handle this error in a specific way and continue to program?

```r
tryCatch(
  error = function(cnd) {
    # code to run when error is thrown
  },
  code_to_run_while_handlers_are_active
)

withCallingHandlers(
  warning = function(cnd) {
    # code to run when warning is signalled
  },
  message = function(cnd) {
    # code to run when message is signalled
  },
  code_to_run_while_handlers_are_active
)
```

- **`tryCatch`** (**exiting** handler) is suitable for working with **Error**  
- **`withCallingHandlers`** (**calling** handler) is suitable for non-error conditions like **Warning** and **Message**

### 4.1 Condition object

In [33]:
pow <- function(x, y) x ^ y

# catch condition
condition_object <- catch_cnd(pow('a', 5))
condition_object

<simpleError in x^y: non-numeric argument to binary operator>

In [34]:
attributes(condition_object)

$names
[1] "message" "call"   

$class
[1] "simpleError" "error"       "condition"

In [37]:
str(condition_object)

List of 2
 $ message: chr "non-numeric argument to binary operator"
 $ call   : language x^y
 - attr(*, "class")= chr [1:3] "simpleError" "error" "condition"


In [38]:
# the message of the condition object
condition_object$message
# or you can use
conditionMessage(condition_object)

[1] "non-numeric argument to binary operator"

[1] "non-numeric argument to binary operator"

In [39]:
# The function call the signal the condition
condition_object$call
# or you can use
conditionCall(condition_object)

x^y

x^y

### 4.2 Exiting handlers

**Exitting handlers**: Once a condition is signaled, it will exit the rest of the function call 

Try-catch-finally

In [48]:
soft_plus <- function(x) {
    tryCatch({
        res <- log(x + 1)
        print('calculate successfully')
        res
    }, error = function(cnd) 'this is the value returned if there was an error')
}


In [49]:
soft_plus(1)

[1] "calculate successfully"


[1] 0.6931472

In [50]:
# Error is signalled, so print('calculate successfully') will not be executed
soft_plus('a')

[1] "this is the value returned if there was an error"

 **`finally`** for clean up

In [51]:
temp <- tempfile()
tryCatch({
    writeLines('HI!', temp)
}, finally = {
    unlink(temp)
})

### 4.3 Calling Handlers

 **Calling handlers**: Once a condition is signalled, it will switch to the handler function, after that, it will switch back to the function call
 
<b style = 'color:red'>NOTE:</b> unlike **`tryCatch`**, **`withCallingHandlers`** return nothing.

In [54]:
withCallingHandlers({
    warning('foo')
}, warning = function(cnd) print('bar'))

[1] "bar"


Warning message in withCallingHandlers({:
"foo"


How this work? When **`warning('foo')`** is executed, it signals a warning condition, so the handler function will be executed, print the word "bar", then after that, it comes back to the function call and print('foo'). You can think of `warning('foo')` as a process of 2 steps:
- 1. It signals warning condition, and this condition will be handled by the hanlder function
- 2. It print the warning messge: "foo"

---
Let's compare **Exitting handler** and **Calling handler**

In [55]:
tryCatch({
    message('one')
    message('two')
}, message = function(cnd) message('three'))

three



once `message('one')` is executed, first, it signals a message condition. Then it will exit the function call (the rest of the code in the function call will be ignored. In this case `message('two')` will not be executed). Because a message condition is signaled, the message condition hanlder function will be executed, so it execute `message('theree')`.

In [56]:
withCallingHandlers({
    message('one')
    message('two')
}, message = function(cnd) message('three'))

three

one

three

two



First, `message('one')` is executed. It signals a message condition, let's call this **signal 1**, so the message condition handler function will be executed, which mean `message('three')` is executed, which signal a message condition (let's cal this **signal 3**), to the environment. If a condition is signaled to the environment, it will be printed. So "three" is printed to the console. After that, **signal 1** from the handler function will be propagated to the environment. so "one" will be printed. The same thing for the code `message('two')`

---
**Muffle** the condition

In [72]:
withCallingHandlers({
    withCallingHandlers({
        message('One')
    }, message = function(cnd) message('two'))
}, message = function(cnd) print('Three'))

[1] "Three"


two



[1] "Three"


One



How does this work?  
First, `message('one')` signal a message condition (call this **signal 1**). This will results in calling the inner handling function that executes `message('two')`, `message('two')` signal a message condition (call this **signal 2**) to the outer handling function, which executes `print('Three')`. After that, **signal 2** from the outer handling function is propagated to the environment, results in printing the message "Two". After that, **signal 1** from the inner handler function will be propagated to the outer handler function, which results in the code `print('Three')`. Then **signal 1** from the outer handler function will be propagated to the environment, which results in printing the message "One".

You can stop the signal of `message('one')` to be propagated to the outer handler function by doing this:

In [73]:
withCallingHandlers({
    withCallingHandlers({
        message('One')
    }, message = function(cnd) {
        message('Two')
        # this function will stop `cnd` to be propagated to the outer handler function
        cnd_muffle(cnd)
    })
}, message = function(cnd) print('Three'))

[1] "Three"


Two



In this case, because **signal 1** is not propagated to the outer handling function and the environment, so the 2 last printting of the previous code will not be executed

### 4.4 Tips and Tricks

>You can use a single handler with this

In [82]:
tryCatch({
   
    warning('Love')
    
}, condition = function(cnd) {
    if(inherits(cnd, 'error')) return('error')
    if(inherits(cnd, 'warning')) return ('warning')
    if(inherits(cnd, 'message')) return ('message')
       
})

[1] "warning"

---
>Try to mimic the **`catch_cnd`** function

In [76]:
catch.condition <- function(exp) {
    tryCatch({
        force(exp)
        return(NULL)
    }, condition = function(cnd) cnd)
}
             

In [77]:
catch.condition(stop('Oops!!'))

<simpleError in force(exp): Oops!!>

In [78]:
catch.condition(warning('Be careful<3'))

<simpleWarning in force(exp): Be careful<3>

In [79]:
catch.condition(message('I love u'))

<simpleMessage in message("I love u"): I love u
>

---
>When to use **`tryCatch`** and **`withCallingHandlers`**?

- if you want to handle error, use **`tryCatch`**
- if you want to handle warning and message, use **`withCallingHandlers`**

---
When using handlers, only **message** and **warning** will be propagated, **error** will not be propagated

In [12]:
# message and warning are stilled printed
withCallingHandlers({
    message('message is propagated to the environment')
    warning('Warning is propagated to the environment', call. = FALSE)
}, message = function(cnd) {
    'do nothing'
}, warning = function(cnd) {
    'do nothing'
})

message is propagated to the environment

Warning message:
"Warning is propagated to the environment"


In [13]:
# no error is printed
tryCatch({
    stop('Fuck off', .call = FALSE)
}, error = function(cnd) 'something wrong')

[1] "something wrong"

---
>Return value of **`tryCatch`** and **`withCallingHandlers`**

- `withCallingHandlers` returns the final value expression if there is no error
- `tryCatch` returns the final value of expression if there is no error, othwerwise it returns the value returned by the error condition handler function

In [22]:
tryCatch({
    # this will return 3 because there is no error in the expression
    1 + 2
}, error = function(cnd) 'something wrong')

[1] 3

In [23]:
# this will return "something wrong"
# because in the expression, we try to add a numeric with a character
tryCatch({
    1 + 'a'
}, error = function(cnd) 'something wrong')

[1] "something wrong"

In [18]:
withCallingHandlers({
    1 + 2
}, condition = function(cnd) 'something is not right')

[1] 3

In [26]:
 withCallingHandlers({
    warning('Broken!!!', call. = F)
}, warning = function(cnd) 'warning condition is signalled')
                           

Warning message:
"Broken!!!"


In [27]:
withCallingHandlers({
    message('Computing....')
}, message = function(cnd) 'wating and be patient')

Computing....



# 5. Custom condition

why custom condition? Because built-in **condition object** only have `message` and `call` which are not much useful

In [87]:
# custom condition object with additional attributes
# class name 'Performance' with 2 additional attributes: "website" and "topic"
custom_cnd_obj <- catch_cnd({
    abort(message = 'Time Limit Exceeded', class = 'performace', website = 'leetcode', topic = 'Graph')
})
str(custom_cnd_obj)

List of 5
 $ message: chr "Time Limit Exceeded"
 $ trace  :List of 3
  ..$ calls  :List of 31
  .. ..$ : language IRkernel::main()
  .. ..$ : language kernel$run()
  .. ..$ : language IRkernel:::handle_shell()
  .. ..$ : language executor$execute(msg)
  .. ..$ : language base::tryCatch(evaluate(request$content$code, envir = .GlobalEnv, output_handler = oh,      stop_on_error = 1L), | __truncated__ ...
  .. ..$ : language base:::tryCatchList(expr, classes, parentenv, handlers)
  .. ..$ : language base:::tryCatchOne(tryCatchList(expr, names[-nh], parentenv, handlers[-nh]),      names[nh], parentenv, handlers[[nh]])
  .. ..$ : language base:::doTryCatch(return(expr), name, parentenv, handler)
  .. ..$ : language base:::tryCatchList(expr, names[-nh], parentenv, handlers[-nh])
  .. ..$ : language base:::tryCatchOne(expr, names, parentenv, handlers[[1L]])
  .. ..$ : language base:::doTryCatch(return(expr), name, parentenv, handler)
  .. ..$ : language evaluate::evaluate(request$content$code,

### 5.1 Signalling

Let's look at the `base::log` function

In [88]:
log('a')

ERROR: Error in log("a"): non-numeric argument to mathematical function


In [89]:
log(1, base = 'a')

ERROR: Error in log(1, base = "a"): non-numeric argument to mathematical function


 let's create a custom signal condition


In [93]:
abort_invalid_argument <- function(argument_name, expected_type) {
    msg <- str_c(argument_name, ' must be of type ', expected_type, ', not ', typeof(argument_name))
    
    abort(message = msg,
         class = 'invalid_argument',
          argument_name = argument_name,
          expected_type = expected_type)
}

In [94]:
# a better log function
my_log <- function(x, base = exp(1)) {
    if(!is.numeric(x))
        abort_invalid_argument('x', 'numeric')
    if(!is.numeric(base))
        abort_invalid_argument('base', 'numeric')
    log(x, base)
}

In [95]:
my_log('a')

ERROR: Error: x must be of type numeric, not character


In [96]:
my_log(1, base = 'a')

ERROR: Error: base must be of type numeric, not character


In [99]:
# catch custom condition object
custom_cnd_obj <- catch_cnd(my_log('a'))
attributes(custom_cnd_obj)

$names
[1] "message"       "trace"         "parent"        "argument_name"
[5] "expected_type"

$class
[1] "invalid_argument" "rlang_error"      "error"            "condition"

### 5.2 Handling

In [103]:
library(testthat)

The following code captures the error, and then asserts it has the structure that we expect.

In [105]:
expect_s3_class(custom_cnd_obj, 'invalid_argument')
expect_equal(custom_cnd_obj$argument_name, 'x')
expect_equal(custom_cnd_obj$expected_type, 'numeric')

Handling custom condition, use `invalid_argument` in `tryCatch`

In [109]:
tryCatch({
    my_log('a')
}, 
         invalid_argument = function(cnd) print('invalid argument'),
         error = function(cnd) print('other error')
)

[1] "invalid argument"


When using **`tryCatch()`** with multiple handlers and custom classes, the first handler to match any class in the signal’s class vector is called, not the best match. For this reason, you need to make sure to put the most specific handlers first. The following code does not do what you might hope:

In [112]:
tryCatch({
    my_log('a')
},
         # error and invalid_arugment both share class "error"
         error = function(cnd) print('other error'),
         invalid_argument = function(cnd) print('invalid argument')
)

[1] "other error"


### 5.3 Exercises

>Inside a package, it’s occasionally useful to check that a package is installed before using it. Write a function that checks if a package is installed (with `requireNamespace("pkg", quietly = FALSE))` and if not, throws a custom condition that includes the package name in the metadata.

In [117]:
abort_uninstalled_package <- function(package_name) {
    abort('missing_package', 
          glue('package "{package_name}" is not installed'),
          package_name = package_name)
}

check_package <- function(name) {
    if(!requireNamespace(name, quietly = T))
        abort_uninstalled_package(name)
}


In [118]:
check_package('ggplot2')

In [119]:
check_package('ggvis')

ERROR: Error: missing_package


# 6. Exercises

>Write **`purrr:possibly`** from scratch

In [10]:
# implement this as an decorator
my_possibly <- function(.f, otherwise, quiet = TRUE) {
  .f <- as_mapper(.f)
  # see section 10.2.3 Forcing Evaluation from this chapter
  # <https://adv-r.hadley.nz/function-factories.html>
  force(otherwise)
  function(...) {
    tryCatch(
      {
        .f(...)
      },
      error = function(cnd) {
        # signal error if we do not want `quiet`
        if (!quiet) {
          message("Error: ", cnd$message)
        }
        otherwise
      },
      interrupt = function(cnd) {
        stop("Terminated by user", .call = FALSE)
      }
    )
  }
}


sum_poss <- my_possibly(sum, 0, FALSE)

sum_poss(1:3)


[1] 6

In [3]:
sum_poss(c(1, 5, 'a'))

Error: invalid 'type' (character) of argument



[1] 0

>Write **`purrr::safely`** from scratch

In [6]:
my_safely <- function(.f, otherwise = NULL, quiet = TRUE) {
  .f <- as_mapper(.f)
  force(otherwise)
  function(...) {
    tryCatch(
      {
        list(result = .f(...), error = NULL)
      },
      error = function(cnd) {
        if (!quiet) {
          message("Error: ", cnd$message)
        }
        list(result = otherwise, error = cnd)
      },
      interrupt = function(cnd) {
        stop("Terminate by user", .call = FALSE)
      }
    )
  }
}


safe_sum <- my_safely(sum, 0, FALSE)

In [8]:
safe_sum(1:10)

$result
[1] 55

$error
NULL

In [9]:
safe_sum(c(1, 'a', 8))

Error: invalid 'type' (character) of argument



$result
[1] 0

$error
<simpleError in .Primitive("sum")(..., na.rm = na.rm): invalid 'type' (character) of argument>
